# Notebook — Clustering para descubrir grupos

### K-means y jerárquico (R, comparando con Python)

En clasificación (como Titanic) teníamos una etiqueta: `Survived`.
En clustering no hay etiqueta: solo tenemos datos y queremos responder algo como:

> “¿Hay grupos naturales aquí dentro?”

Eso sirve para segmentación de clientes, detección de comportamientos, agrupación de sensores, etc.

Hoy vamos a hacer dos métodos clásicos:

* **K-means** (rápido, pero hay que decir cuántos grupos)
* **Jerárquico** (exploratorio, se ve como un árbol)

Y en ambos casos haremos algo que en IA es obligatorio: **escalar** los datos.

---

## Preparar el entorno

En Python normalmente haríamos:

* `pandas` para datos
* `StandardScaler`, `KMeans` y alguna función de `scipy` para jerárquico

En R:

* `tidyverse` para datos y gráficos
* `cluster` para silhouette
* `stats` (viene con R) para `kmeans`, `dist`, `hclust`


In [ ]:
library(tidyverse)  # Para manipulación de datos
library(cluster)    # Algoritmos base
library(factoextra) # 🚀 Para visualización y automatización

---

## Cargamos los datos

Vamos a usar un dataset sencillo de clientes con dos variables:

* ingresos
* gasto anual

In [ ]:
clientes <- read_csv(
  "https://gist.githubusercontent.com/pravalliyaram/5c05f43d2351249927b8a3f3cc3e5ecf/raw/8bd6144a87988213693754baaa13fb204933282d/Mall_Customers.csv"
)

En Python habríamos hecho:

```python
pd.read_csv("mall_customers.csv")

Vemos que las columnas importantes están con nombres “incómodos”:

* `Annual Income (k$)`
* `Spending Score (1-100)`

En Python esto suele ser más tolerable (pero también molesto).
En R se puede usar con comillas invertidas, pero para un cuaderno didáctico es mejor renombrar.

---

## Renombrar para trabajar cómodos

Esta parte es muy práctica para que el alumnado aprenda “higiene de datos”.

In [ ]:
clientes <- clientes %>%
  rename(
    Annual_Income = `Annual Income (k$)`,
    Spending_Score = `Spending Score (1-100)`
  )
glimpse(clientes)

Ahora ya podemos usar `Annual_Income` y `Spending_Score` sin dolores.


## Primer vistazo a los datos

Como siempre, antes de hacer IA, miramos los datos.

En Python usábamos `info()`.
En R usamos `glimpse()`.

In [ ]:
glimpse(clientes)

Y un gráfico:


In [ ]:
ggplot(clientes, aes(x = Annual_Income, y = Spending_Score)) +
  geom_point(alpha = 0.7)

##  Elegir variables (y por qué)

Para clustering necesitamos columnas numéricas.
Aquí vamos a empezar con una pregunta sencilla:

> “¿Se pueden agrupar clientes por ingresos y gasto?”

Por eso usamos solo:

- Annual_Income
- Spending_Score

In [ ]:
datos <- clientes %>% select(Annual_Income, Spending_Score)


---

## Una advertencia importante antes de hacer clustering

En Python ya vimos que **K-means depende mucho de la escala** de los datos.
Si una variable tiene valores mucho más grandes que otra, domina el resultado.

Por eso, antes de hacer clustering, **escalamos los datos**.

En Python usábamos `StandardScaler`.
En R usamos `scale()`.

In [ ]:
datos_scaled <- scale(datos)


# K-means

## Elegir el número de clusters: “método del codo”

En Python normalmente:

* entrenamos K-means con distintos k
* miramos la “inercia”
* elegimos el “codo”

En Python, para elegir el número de grupos ($k$), se suele hacer un bucle `for` para calcular la inercia.
En R con `factoextra`, es una sola línea

In [ ]:
# Gráfico del Codo (Elbow Method) automático
# Busca dónde la curva deja de bajar bruscamente
fviz_nbclust(datos_scaled, kmeans, method = "wss") +
  labs(title = "Método del Codo")

# Gráfico de la Silueta (Silhouette) automático
# Busca la barra más alta (mejor separación)
fviz_nbclust(datos_scaled, kmeans, method = "silhouette") +
  labs(title = "Método de la Silueta")

No hay un “k perfecto”, pero buscamos un punto donde a partir de ahí mejorar cuesta mucho.


---

## Entrenar K-means con un k elegido

Parece que $k=5$ es un buen número según los gráficos.Python: `KMeans(n_clusters=5).fit(X)`

In [ ]:
set.seed(123) # Para resultados reproducibles
km_model <- kmeans(datos_scaled, centers = 5, nstart = 25)

# Ver tamaños de los grupos
km_model$size

Aquí es donde R brilla. En Python pintar las elipses de confianza requiere mucho código. Aquí:

In [ ]:
fviz_cluster(km_model, data = datos_scaled,
             palette = "jco",         # Paleta de colores académica
             geom = "point",          # Solo puntos (quita 'text' si se ve sucio)
             ellipse.type = "convex", # Dibuja el contorno del grupo
             ggtheme = theme_minimal())

Observa cómo detecta perfectamente los grupos: bajo ingreso/bajo gasto, bajo ingreso/alto gasto, etc.

Aquí ya podemos empezar a hablar de perfiles:
clientes con ingresos altos y gasto bajo, etc.

---

# Clustering jerárquico

A diferencia de K-Means, que intenta encontrar centros, el clustering jerárquico conecta puntos basándose en su cercanía. Es un proceso de tres pasos obligatorios:

## Medir distancias

¿Cuán lejos está cada cliente de todos los demás?


In [ ]:
# 1. Matriz de distancias
distancias <- dist(datos_scaled, method = "euclidean")



## Agrupar (Linkage)

Unir los puntos más cercanos poco a poco hasta formar un gran árbol.

In [ ]:
# 2. Crear el árbol (Linkage)
hc_model <- hclust(distancias, method = "ward.D2")



## Cortar

Decidir dónde podar el árbol para quedarnos con los grupos finales.

In [ ]:
# 3. Visualizar Dendrograma coloreado
fviz_dend(hc_model, k = 5,  # Cortamos en 5 grupos
          cex = 0.5,        # Tamaño de texto pequeño
          k_colors = "jco",
          rect = TRUE,      # Recuadros alrededor de grupos
          main = "Dendrograma de Clientes")

---

## Interpretación de Negocio (Lo más importante)

El algoritmo dice "Grupo 1". Tu trabajo es decir "Clientes VIP". Para esto, volvemos a unir los clusters con los datos originales (¡sin escalar!) y usamos dplyr.

In [ ]:
clientes_final %>%
  group_by(cluster) %>%
  summarise(
    n_clientes = n(),
    # Usamos los nombres en inglés/originales
    ingresos_promedio = mean(Annual_Income), 
    gasto_promedio = mean(Spending_Score)
  ) %>%
  arrange(desc(ingresos_promedio))

### 🧠 Resumen: Python vs R (Clustering)

| Tarea | Python (Scikit-Learn/Matplotlib) | R (Factoextra/Tidyverse) |
| --- | --- | --- |
| **Escalar** | `StandardScaler().fit_transform()` | `scale()` |
| **Codo (Elbow)** | Bucle `for` + lista de inercia | `fviz_nbclust(..., method="wss")` |
| **Modelo** | `KMeans(n_clusters=k)` | `kmeans(centers=k)` |
| **Gráfico Cluster** | `plt.scatter` (manual colores) | `fviz_cluster` (elipses auto) |
| **Dendrograma** | `scipy.cluster.hierarchy` | `fviz_dend` |
| **Perfiles** | `df.groupby()` | `df %>% group_by() %>% summarise()` |


# Ejercicio

Ahora vas a repetir el mismo flujo con un dataset diferente, como hicimos antes con Housing.

Vamos a usar un dataset muy conocido: **Iris** (clásico de ML).
En Python lo habrás visto muchas veces; en R viene incluido.

La idea:

* elegir dos variables numéricas
* escalar
* K-means + silhouette
* jerárquico + dendrograma
* comparar

## Cargar Iris

In [ ]:
data(iris)
glimpse(iris)

Nos quedamos con dos columnas (para poder visualizar fácil):

* `Sepal.Length`
* `Petal.Length`

In [ ]:
iris2 <- iris %>% select(Sepal.Length, Petal.Length)
iris2_scaled <- scale(iris2)

## K-means + silhouette


In [ ]:
set.seed(123)
km_iris <- kmeans(iris2_scaled, centers = 3, nstart = 25)

sil_iris <- silhouette(km_iris$cluster, dist(iris2_scaled))
mean(sil_iris[, 3])
plot(sil_iris, border = NA)

Visualiza:


In [ ]:
iris_plot <- iris %>%
  mutate(cluster_kmeans = factor(km_iris$cluster))

ggplot(iris_plot, aes(x = Sepal.Length, y = Petal.Length, color = cluster_kmeans)) +
  geom_point(size = 2, alpha = 0.8)

## Jerárquico + corte


In [ ]:
hc_iris <- hclust(dist(iris2_scaled), method = "ward.D2")
plot(hc_iris, labels = FALSE, main = "Dendrograma Iris")

iris_plot$cluster_hc <- factor(cutree(hc_iris, k = 3))

ggplot(iris_plot, aes(x = Sepal.Length, y = Petal.Length, color = cluster_hc)) +
  geom_point(size = 2, alpha = 0.8)

### Preguntas de interpretación

1. ¿Cuál de los dos métodos separa mejor los grupos?
2. ¿Qué silhouette medio te sale con k = 2, 3 y 4?
3. Si tuvieras que explicar los clusters en palabras, ¿cómo los describirías?

---

## Chuleta Python ↔ R (clustering)

| Lo que hacemos   | Python               | R               |
| ---------------- | -------------------- | --------------- |
| Escalar          | `StandardScaler()`   | `scale()`       |
| K-means          | `KMeans()`           | `kmeans()`      |
| Etiquetas        | `labels_`            | `$cluster`      |
| Elbow            | `inertia_`           | `tot.withinss`  |
| Silhouette score | `silhouette_score()` | `mean(sil[,3])` |
| Silhouette plot  | (varía)              | `plot(sil)`     |
| Distancias       | `scipy` / `sklearn`  | `dist()`        |
| Jerárquico       | `linkage()`          | `hclust()`      |
| Cortar árbol     | `fcluster()`         | `cutree()`      |